# Keras

## Data Preparation and Processing

In [44]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [4]:
train_labels = []
train_samples = []

Example data:

- An experimental drug tested in individuals from ages 13 to 100 in a clinical trial.
- The trial had 2100 participants. Half were uder 65 years old, half were 65 years or older.
- Around 95% of patients 65 or older experienced side effects.
- Around 95% of patients under 65 experienced no side effects.

We are **generating** our data sets:

In [6]:
for i in range(50):
    # The ~5% of younger individuals who did experience side effects
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)
    
    # The ~5% of older individuals who did not experience side effects
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # The ~95% of younger individuals who did not experience side effects
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)
    
    # The ~95% of older individuals who did experience side effects
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

Summary of our data:

In [31]:
[print(j,train_labels[i]) for i, j in enumerate(train_samples)]

25 1
33 1
68 0
15 1
72 0
55 1
89 0
58 1
71 0
39 1
82 0
32 1
90 0
50 1
72 0
46 1
100 0
48 1
94 0
27 1
89 0
51 1
95 0
57 1
100 0
57 1
98 0
61 1
75 0
35 1
83 0
45 1
70 0
60 1
71 0
46 1
66 0
24 1
84 0
57 1
98 0
21 1
100 0
39 1
76 0
14 1
94 0
32 1
91 0
39 1
99 0
31 1
82 0
44 1
73 0
61 1
72 0
58 1
70 0
13 1
91 0
54 1
100 0
56 1
74 0
38 1
79 0
45 1
99 0
55 1
97 0
36 1
84 0
63 1
94 0
24 1
98 0
20 1
100 0
39 1
67 0
47 1
75 0
62 1
73 0
24 1
79 0
41 1
79 0
21 1
80 0
51 1
87 0
37 1
80 0
58 1
84 0
50 1
75 0
14 1
89 0
55 0
69 1
52 0
75 1
52 0
94 1
14 0
87 1
57 0
100 1
16 0
94 1
48 0
80 1
45 0
69 1
20 0
69 1
60 0
78 1
52 0
99 1
50 0
88 1
55 0
70 1
53 0
77 1
32 0
73 1
35 0
84 1
39 0
75 1
38 0
98 1
16 0
99 1
20 0
93 1
35 0
95 1
57 0
77 1
27 0
98 1
40 0
87 1
15 0
81 1
16 0
76 1
31 0
85 1
53 0
68 1
41 0
85 1
21 0
79 1
13 0
69 1
15 0
68 1
48 0
93 1
20 0
93 1
33 0
70 1
42 0
70 1
27 0
92 1
35 0
93 1
62 0
94 1
31 0
68 1
18 0
77 1
20 0
100 1
16 0
98 1
45 0
77 1
42 0
72 1
31 0
76 1
47 0
72 1
62 0
86 1
59 0
72 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Next, we **process** our data: (to be in the format required by the `fit()` function which `numpy array`.)

In [38]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels, train_samples = shuffle(train_labels, train_samples)

The shuffle stage is performed to get rid of any imposed order from the data generation process!

In [41]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

The reshape is a foramlity, since `fit()` do not accept 1-dimensional data!

In [42]:
for i in scaled_train_samples:
    print(i)

[0.10344828]
[0.91954023]
[0.10344828]
[0.45977011]
[0.43678161]
[0.63218391]
[0.31034483]
[0.83908046]
[0.1954023]
[0.91954023]
[0.08045977]
[0.09195402]
[0.98850575]
[0.89655172]
[0.29885057]
[0.75862069]
[0.02298851]
[0.8045977]
[0.64367816]
[0.52873563]
[0.]
[0.98850575]
[0.86206897]
[0.98850575]
[0.03448276]
[0.67816092]
[0.2183908]
[0.20689655]
[0.20689655]
[0.36781609]
[0.74712644]
[0.86206897]
[0.45977011]
[0.81609195]
[0.03448276]
[0.74712644]
[0.86206897]
[0.64367816]
[0.18390805]
[0.67816092]
[0.90804598]
[1.]
[0.98850575]
[0.50574713]
[0.85057471]
[0.65517241]
[0.35632184]
[0.71264368]
[0.33333333]
[0.74712644]
[0.42528736]
[0.4137931]
[0.03448276]
[0.17241379]
[0.67816092]
[0.26436782]
[0.42528736]
[0.35632184]
[0.66666667]
[0.44827586]
[0.20689655]
[0.73563218]
[0.87356322]
[0.13793103]
[0.71264368]
[0.7816092]
[0.71264368]
[0.74712644]
[0.82758621]
[0.1954023]
[0.91954023]
[0.25287356]
[0.89655172]
[0.86206897]
[0.7816092]
[0.83908046]
[0.8045977]
[0.12643678]
[0.2413793

[0.87356322]
[0.8045977]
[0.18390805]
[0.22988506]
[0.55172414]
[0.44827586]
[0.57471264]
[0.96551724]
[0.85057471]
[0.20689655]
[0.88505747]
[0.18390805]
[0.08045977]
[0.37931034]
[0.42528736]
[0.35632184]
[1.]
[0.29885057]
[0.8045977]
[0.18390805]
[0.32183908]
[0.63218391]
[0.12643678]
[0.86206897]
[0.65517241]
[0.81609195]
[0.20689655]
[0.52873563]
[0.70114943]
[0.74712644]
[0.08045977]
[0.88505747]
[0.72413793]
[0.11494253]
[0.16091954]
[0.93103448]
[0.97701149]
[0.71264368]
[0.43678161]
[0.33333333]
[0.51724138]
[0.90804598]
[0.66666667]
[0.08045977]
[0.45977011]
[0.94252874]
[0.54022989]
[0.2183908]
[0.86206897]
[0.]
[0.66666667]
[0.95402299]
[0.26436782]
[0.67816092]
[0.33333333]
[0.72413793]
[0.55172414]
[0.98850575]
[0.70114943]
[0.24137931]
[0.64367816]
[0.5862069]
[0.45977011]
[0.28735632]
[0.32183908]
[0.2183908]
[0.81609195]
[0.67816092]
[0.75862069]
[0.10344828]
[0.49425287]
[0.7816092]
[0.91954023]
[0.71264368]
[0.85057471]
[0.77011494]
[0.64367816]
[0.85057471]
[0.62068

[0.98850575]
[1.]
[0.13793103]
[0.65517241]
[0.81609195]
[0.93103448]
[0.88505747]
[0.71264368]
[0.50574713]
[0.96551724]
[0.4137931]
[0.68965517]
[0.20689655]
[0.6091954]
[0.26436782]
[0.31034483]
[0.73563218]
[0.37931034]
[0.57471264]
[0.01149425]
[0.66666667]
[0.27586207]
[0.09195402]
[0.22988506]
[0.74712644]
[0.40229885]
[0.94252874]
[0.28735632]
[0.48275862]
[0.97701149]
[0.72413793]
[0.96551724]
[0.09195402]
[0.87356322]
[0.56321839]
[0.72413793]
[0.50574713]
[0.09195402]
[0.49425287]
[0.06896552]
[0.06896552]
[0.32183908]
[0.86206897]
[0.4137931]
[0.34482759]
[0.43678161]
[0.77011494]
[0.54022989]
[0.22988506]


## Simple tf.keras Sequential Model

Now we create artificial neural network for which we can train on this data: 

In [2]:
import tensorflow as tf
from tensorflow import keras